In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [4]:
## Removing observations with NA
heart = heart.dropna()

In [5]:
## Defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [7]:
## Building the logisitic model 
logit_md = LogisticRegression().fit(X_train, Y_train)

## Predicting on the test dataset
logit_pred = logit_md.predict_proba(X_test)[:, 1]
logit_pred

array([0.06258583, 0.13767685, 0.28665817, 0.03054755, 0.19287944,
       0.3163654 , 0.18360572, 0.30758725, 0.12866084, 0.30234502,
       0.26741228, 0.08767922, 0.26212235, 0.16117155, 0.09994736,
       0.29500248, 0.08613389, 0.22826128, 0.29499256, 0.31588885,
       0.07344404, 0.04986988, 0.07923232, 0.09929359, 0.275017  ,
       0.03043076, 0.0866084 , 0.03479606, 0.05493478, 0.09495167,
       0.13776988, 0.14330194, 0.14374947, 0.08303473, 0.18596503,
       0.11681107, 0.194653  , 0.2453331 , 0.33804128, 0.05162088,
       0.16955067, 0.15094852, 0.07542171, 0.07988906, 0.11993224,
       0.11591126, 0.15128396, 0.09559905, 0.08240087, 0.0878569 ,
       0.23980422, 0.22753312, 0.20481304, 0.09040743, 0.10529344,
       0.1273893 , 0.09632899, 0.03983882, 0.05872526, 0.11639581,
       0.12146111, 0.08623983, 0.13203013, 0.15555604, 0.07904435,
       0.05538985, 0.09176462, 0.04078562, 0.25496921, 0.05780606,
       0.19224663, 0.19373586, 0.03438921, 0.10023903, 0.07568

In [8]:
## Changing likelihoods to labels
logit_label = np.where(logit_pred < 0.25, 0, 1)
logit_label

array([0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
## Constructing the confusion matrix
confusion_matrix(Y_test, logit_label)

array([[536,  87],
       [ 75,  34]])

In [10]:
## Computing the accuracy
accuracy_score(Y_test, logit_label)

0.7786885245901639